In [1]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [2]:
num_qubits = 2
max_weight = 2
term_dict = {("S", "XX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [3]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    intersect = used_indices_1.intersection(used_indices_2)
    if len(intersect) > 0 and len(intersect) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
measure_string_attributes = list([p for p in measure_string_iterator])

hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)



import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[1:] for s in el[-4:-1])
            egen = ",".join(str(s) for s in el[:-4])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])
ic(df)

ic| df:           H,+X_  H,+Y_  H,+Z_  H,+_X  H,+_Y  H,+_Z  H,+XX  H,+XY  H,+XZ  \
        XX,XX,X_      0      0      0      0      0      0      0      0      0   
        XX,XX,_X      0      0      0      0      0      0      0      0      0   
        XX,XX,XX      0      0      0      0      0      0      0      0      0   
        XX,XY,X_      0      0      0      0      0      0      0      0      0   
        XX,XY,_Y      0      0      0      0      0      2      0      0      2   
        ...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
        ZZ,ZY,_Y      0      0      0     -2      0      0      0      0      0   
        ZZ,ZY,ZY      0      0      0     -2      0      0      0      0      0   
        ZZ,ZZ,Z_      0      0      0      0      0      0      0      0      0   
        ZZ,ZZ,_Z      0      0      0      0      0      0      0      0      0   
        ZZ,ZZ,ZZ      0      0      0      0      0      0      0      0      0   
    

,"H,+X_","H,+Y_","H,+Z_","H,+_X","H,+_Y","H,+_Z","H,+XX","H,+XY","H,+XZ","H,+YX",...,"A,+YY,+YZ","A,+YY,+ZX","A,+YY,+ZY","A,+YY,+ZZ","A,+YZ,+ZX","A,+YZ,+ZY","A,+YZ,+ZZ","A,+ZX,+ZY","A,+ZX,+ZZ","A,+ZY,+ZZ"
"XX,XX,X_",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"XX,XX,_X",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"XX,XX,XX",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"XX,XY,X_",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"XX,XY,_Y",0,0,0,0,0,2,0,0,2,0,...,0,2,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"ZZ,ZY,_Y",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
"ZZ,ZY,ZY",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
"ZZ,ZZ,Z_",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"ZZ,ZZ,_Z",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-4,0,0


In [4]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [5]:
import pygsti
from pygsti.extras import idletomography as idt
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1
    )

mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")
updated_ckt_list = []
from pygsti.baseobjs import Label
for ckt in idle_experiments:
    new_ckt = ckt.copy(editable=True)
    for i, lbl in enumerate(ckt):
        if lbl == Label(()):
            # new_ckt[i] = [Label(("Gi", i)) for i in range(n_qubits)]
            new_ckt[i] = Label("Gi", tuple(i for i in range(num_qubits)))
            # new_ckt[i] = Label(("Gi", 0))
    updated_ckt_list.append(new_ckt)

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, updated_ckt_list, 10000000, seed=8675309, sample_error="none"
)
ds

C:\Users\rundx\pyGSTi\pygsti\circuits\circuit.py:816: UserWarning: Editable circuit is being converted to read-only mode in order to hash it.  You should call circuit.done_editing() beforehand.
  _warnings.warn(("Editable circuit is being converted to read-only"


In [6]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=((),),):
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    # print(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_observables = all_full_length_observables(
            pauli_fidpair[1], nqubits
        )
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            #info["jacobian row"] = full_jacobian[ifp]
            infos_for_this_fidpair[out] = info
            # ic(infos_for_this_fidpair)
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("+","").replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
            # obs_err_rates = np.concatenate([np.array([
            #                 observed_error_rates[i]
            #                 for i in range(len(all_fidpairs))
            #                 ]
            #             )
            #         ]
            #     )
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [7]:
idl_string = "Gi:"+":".join((str(i) for i in range(num_qubits)))
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts, idle_string=idl_string)

In [8]:
observed_error_rates

{0: [-1.6953455232808294e-16,
  -1.6953455232808294e-16,
  -1.6953455232808294e-16],
 1: [-1.6953455232808294e-16, 0.0, 0.0],
 2: [-1.6953455232808294e-16, 0.0, 0.0],
 3: [0.0, -1.6953455232808294e-16, 0.0],
 4: [0.0, 0.0, 0.0],
 5: [0.0, 0.0, 0.0],
 6: [0.0, -1.6953455232808294e-16, 0.0],
 7: [0.0, 0.0, 0.0],
 8: [0.0, 0.0, 0.0],
 9: [-1.6953455232808294e-16, 0.0, 0.0],
 10: [-3.3948691372684573e-16,
  -0.00019994002864204022,
  -0.00019994002864204022],
 11: [-1.6953455232808294e-16, 0.0, 0.0],
 12: [0.0, 0.0, 0.0],
 13: [0.0, -0.00019994002864204022, 0.0],
 14: [0.0, 0.0, 0.0],
 15: [0.0, 0.0, 0.0],
 16: [0.0, -0.00019994002864204022, 0.0],
 17: [0.0, 0.0, 0.0],
 18: [-1.6953455232808294e-16, 0.0, 0.0],
 19: [-1.6953455232808294e-16, 0.0, 0.0],
 20: [-3.3948691372684573e-16,
  -0.00019994002864204022,
  -0.00019994002864204022],
 21: [0.0, 0.0, 0.0],
 22: [0.0, 0.0, 0.0],
 23: [0.0, -0.00019994002864204022, 0.0],
 24: [0.0, 0.0, 0.0],
 25: [0.0, 0.0, 0.0],
 26: [0.0, -0.000199940028

In [9]:
obs_error_rates_by_exp

{('XX', 'XX', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XX', ' _X'): -1.6953455232808294e-16,
 ('XX', 'XX', ' XX'): -1.6953455232808294e-16,
 ('XX', 'XY', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XY', ' _Y'): 0.0,
 ('XX', 'XY', ' XY'): 0.0,
 ('XX', 'XZ', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XZ', ' _Z'): 0.0,
 ('XX', 'XZ', ' XZ'): 0.0,
 ('XX', 'YX', ' Y_'): 0.0,
 ('XX', 'YX', ' _X'): -1.6953455232808294e-16,
 ('XX', 'YX', ' YX'): 0.0,
 ('XX', 'YY', ' Y_'): 0.0,
 ('XX', 'YY', ' _Y'): 0.0,
 ('XX', 'YY', ' YY'): 0.0,
 ('XX', 'YZ', ' Y_'): 0.0,
 ('XX', 'YZ', ' _Z'): 0.0,
 ('XX', 'YZ', ' YZ'): 0.0,
 ('XX', 'ZX', ' Z_'): 0.0,
 ('XX', 'ZX', ' _X'): -1.6953455232808294e-16,
 ('XX', 'ZX', ' ZX'): 0.0,
 ('XX', 'ZY', ' Z_'): 0.0,
 ('XX', 'ZY', ' _Y'): 0.0,
 ('XX', 'ZY', ' ZY'): 0.0,
 ('XX', 'ZZ', ' Z_'): 0.0,
 ('XX', 'ZZ', ' _Z'): 0.0,
 ('XX', 'ZZ', ' ZZ'): 0.0,
 ('XY', 'XX', ' X_'): -1.6953455232808294e-16,
 ('XY', 'XX', ' _X'): 0.0,
 ('XY', 'XX', ' XX'): 0.0,
 ('XY', 'XY', ' X_'): -3.39486

In [10]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [11]:
j = df.to_numpy()

In [12]:
jinv = np.linalg.pinv(j)

In [13]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [14]:
dict(zip(df.columns,intrins_errs))

{'H,+X_': 6.776263578034403e-21,
 'H,+Y_': -3.527175104365612e-07,
 'H,+Z_': 3.5271751043656075e-07,
 'H,+_X': 4.0234064994579266e-21,
 'H,+_Y': -3.5271751043657647e-07,
 'H,+_Z': 3.5271751043658055e-07,
 'H,+XX': -1.3552527156068805e-20,
 'H,+XY': -2.683088903304623e-06,
 'H,+XZ': 2.6830889033046226e-06,
 'H,+YX': -2.6830889033046298e-06,
 'H,+YY': 1.2434099170884807e-06,
 'H,+YZ': 2.541098841762901e-21,
 'H,+ZX': 2.6830889033046196e-06,
 'H,+ZY': 1.0587911840678754e-21,
 'H,+ZZ': -1.2434099170884852e-06,
 'S,+X_': 3.504880013948145e-06,
 'S,+Y_': 3.197584605990773e-06,
 'S,+Z_': 3.19758460599077e-06,
 'S,+_X': 3.5048800139481445e-06,
 'S,+_Y': 3.1975846059907732e-06,
 'S,+_Z': 3.1975846059907766e-06,
 'S,+XX': 4.212375470998e-05,
 'S,+XY': 2.335009671266023e-06,
 'S,+XZ': 2.335009671266037e-06,
 'S,+YX': 2.3350096712660333e-06,
 'S,+YY': 2.9327896207203915e-06,
 'S,+YZ': 2.155191064991238e-06,
 'S,+ZX': 2.3350096712660252e-06,
 'S,+ZY': 2.1551910649912622e-06,
 'S,+ZZ': 2.93278962072